<div style="text-align: center">
  <img src="https://github.com/KarolisRam/MineRL2021-Intro-baselines/blob/main/img/colab_banner.png?raw=true">
</div>

# Introduction
This notebook is part one of the Intro track baselines for the [MineRL 2021](https://minerl.io/) competition. It provides an easy way to try out the MineRL environment without having to install anything on your machine. The notebook should run in 5-10 minutes if you click `Runtime -> Run All` in the menu above.  

If you want to edit the notebook, you will have to save a copy to your Google Drive. Simply click `File -> Save a copy in Drive`.

Below you will find a fully scripted agent that spawns in a fixed world and executes a sequence of actions to acquire 6 pieces of wood, craft a wooden pickaxe and dig some cobblestone.  

You can try adjusting the scripted part to progress further in the task and acquire more rewards. Please note that in the competition setting your agent will spawn in a random world - this is explored in the second notebook:  

[MineRL fully scripted](https://colab.research.google.com/drive/1ipj34U_Ub8IsTO0I80o4bUTtltERMErm?usp=sharing)

# Setup

In [1]:
# %%capture
# ^ hides output
# !sudo add-apt-repository -y ppa:openjdk-r/ppa
# !sudo apt-get purge openjdk-*
# !sudo apt-get install openjdk-8-jdk
!sudo apt-get install xvfb
# !sudo apt-get install xserver-xephyr vnc4server python-opengl ffmpeg

Identified face as cyprien
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xvfb is already the newest version (2:21.1.3-2ubuntu2.3).
The following packages were automatically installed and are no longer required:
  libasound2-dev libflashrom1 libftdi1-2 libportaudiocpp0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [2]:
# %%capture
# ^ hides output
# !pip3 install --upgrade minerl

In [3]:
# %%capture
# Install for notebook use
!pip3 install pyvirtualdisplay
!pip3 install -U colabgymrender

  Using cached PyVirtualDisplay-3.0-py3-none-any.whl (15 kB)
  Using cached colabgymrender-1.1.0.tar.gz (3.5 kB)
  Preparing metadata (setup.py) ... done
  Using cached moviepy-1.0.3.tar.gz (388 kB)
  Preparing metadata (setup.py) ... done
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Using cached proglog-0.1.10-py3-none-any.whl (6.1 kB)
  Created wheel for colabgymrender: filename=colabgymrender-1.1.0-py3-none-any.whl size=3114 sha256=3975961d6c007dac7d64c512fd5d0f8f7332cda90a1c11e73cb0a1b9d915c139
  Stored in directory: /home/cyprien/.cache/pip/wheels/98/27/91/37850f9c1ea024629bd9d2b2374cd511f47d1620ef2a947949
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110727 sha256=e7e2bc942a54f32ad32576e4173f086345ccc5ee843aa31c523dbc78c0aeb141
  Stored in directory: /home/cyprien/.cache/pip/wheels/05/c3/60/ff37aa492e097b78c495e049d4ec5dbd5be98bd9b36eb1f0b1
Successfully built colabgymrender moviepy
  Attempting uninstall: decorator
    Found existing 

# Import libraries

In [1]:
import sys
sys.path.append("../")

import gym
import minerl
from tqdm.notebook import tqdm
from colabgymrender.recorder import Recorder
from pyvirtualdisplay import Display
import logging
from src.utils import str_to_act

logging.disable(logging.ERROR) # reduce clutter, remove if something doesn't work to see the error logs.

# Start of the agent code

# Actions
Here's a list of all possible actions:
```
Dict(attack:Discrete(2),
     back:Discrete(2),
     camera:Box(low=-180.0, high=180.0, shape=(2,)),
     craft:Enum(crafting_table,none,planks,stick,torch),
     equip:Enum(air,iron_axe,iron_pickaxe,none,stone_axe,stone_pickaxe,wooden_axe,wooden_pickaxe),
     forward:Discrete(2),
     jump:Discrete(2),
     left:Discrete(2),
     nearbyCraft:Enum(furnace,iron_axe,iron_pickaxe,none,stone_axe,stone_pickaxe,wooden_axe,wooden_pickaxe),
     nearbySmelt:Enum(coal,iron_ingot,none),
     place:Enum(cobblestone,crafting_table,dirt,furnace,none,stone,torch),
     right:Discrete(2),
     sneak:Discrete(2),
     sprint:Discrete(2))
```

### Camera
Camera actions contain two values:
1. Pitch (up/down), where up is negative, down is positive.
2. Yaw (left/right), where left is negative, right is positive.  

For example, moving the camera up by 10 degrees would be 'camera:[-10,0]'.


# Change agent behaviour here
To change the sequence of actions that the agent performs, change the code inside the `get_action_sequence()` function below. One action is done every tick and there are 20 ticks per second in a regular Minecraft game.  


In [2]:
def get_action_sequence():
    """
    Specify the action sequence for the agent to execute.
    """
    # get 6 logs:
    action_sequence = []
    action_sequence += [''] * 100  # wait 5 sec
    action_sequence += ['forward'] * 8
    action_sequence += ['attack'] * 61
    action_sequence += ['camera:[-10,0]'] * 7  # look up
    action_sequence += ['attack'] * 61
    action_sequence += ['attack'] * 61
    action_sequence += ['attack'] * 61
    action_sequence += ['attack'] * 61
    action_sequence += [''] * 50
    action_sequence += ['jump']
    action_sequence += ['forward'] * 10
    action_sequence += ['camera:[-10,0]'] * 2
    action_sequence += ['attack'] * 61
    action_sequence += ['attack'] * 61
    action_sequence += ['attack'] * 61
    action_sequence += ['camera:[10,0]'] * 9  # look down
    action_sequence += [''] * 50

    # make planks, sticks, crafting table and wooden pickaxe:
    action_sequence += ['back'] * 2
    action_sequence += ['craft:planks'] * 4
    action_sequence += ['craft:stick'] * 2
    action_sequence += ['craft:crafting_table']
    action_sequence += ['camera:[10,0]'] * 9
    action_sequence += ['jump']
    action_sequence += [''] * 5
    action_sequence += ['place:crafting_table']
    action_sequence += [''] * 10

    # bug: looking straight down at a crafting table doesn't let you craft. So we look up a bit before crafting:
    action_sequence += ['camera:[-1,0]']
    action_sequence += ['nearbyCraft:wooden_pickaxe']
    action_sequence += ['camera:[1,0]']
    action_sequence += [''] * 10
    action_sequence += ['equip:wooden_pickaxe']
    action_sequence += [''] * 10

    # dig down:
    action_sequence += ['attack'] * 600
    action_sequence += [''] * 10

    return action_sequence

# Start Minecraft

In [3]:
display = Display(visible=False, size=(400, 300))
display.start();

In [4]:
# env = gym.make('MineRLObtainDiamond-v0')
env = gym.make('MineRLBasaltFindCave-v0')
env = Recorder(env, './video', fps=60)

/home/cyprien/anaconda3/envs/mc/lib/python3.9/runpy.py:127: RuntimeWarning: 'minerl.utils.process_watcher' found in sys.modules after import of package 'minerl.utils', but prior to execution of 'minerl.utils.process_watcher'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))


# Run your agent
After the code below finishes you should see a video of the agent and a line saying it received 35.0 reward.

In [5]:
action_sequence = get_action_sequence()

env.seed(21)
obs = env.reset();
 
total_reward = 0
for i, action in enumerate(tqdm(action_sequence)):
    obs, reward, done, _ = env.step(str_to_act(env, action))
    # if reward:
    #     print(f"action executed: {action}")
    #     print(f"reward: {reward}")
    total_reward += reward
    if done:
        break

env.release()
env.play()  # Need to change progress_bar= to debug=None in lib files (recorder.py, play function)

print(f'\nTotal reward = {total_reward}')

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


  0%|          | 0/1394 [00:00<?, ?it/s]

TypeError: write_videofile() got an unexpected keyword argument 'progress_bar'